#  SUSY searches using Machine Learning  

In [1]:
%matplotlib inline
import numpy as np
import sys
import math
from sklearn.preprocessing import Imputer
import os.path
import h5py
import matplotlib.pyplot as plt

/scratch2/eirikgr/egramsta/postdoc/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from ROOT import gROOT, gDirectory, TFile, TEventList, TCut, TH1F, TH2F, TCanvas, kRed
from root_numpy import tree2array, root2array
%jsroot on

ImportError: No module named ROOT

We need to define som function to convert the ROOT nTuples into numpy arrays. One possibility is to read from the nTuple each time and make the numpy array. This can be done by using the following function (buildArraysFromROOT()). It takes as input a TTree, variables to print, selection criteria, which events to write and finally name of the selection.

In [ ]:
def buildArraysFromROOT(tree,allowedFeatures,cut,skipEvents,maxEvents,name):
    dataContainer = {}
    featureNames = []
    eventCounter = -1
    gROOT.Reset()
    
    # Get branch names
    for item in tree.GetListOfBranches():
        featureName = item.GetName()
        if featureName in allowedFeatures:
            featureNames.append(featureName)
            dataContainer[featureName] = []

    # Build the event list
    tcut = TCut(cut)
    tree.Draw(">>eventList",tcut)
    eventList = TEventList()
    eventList = gDirectory.Get("eventList")
    nSelectedEvents = eventList.GetN()

    # Event loop
    for i in range(0,nSelectedEvents):
        if (i < skipEvents):
            continue
        if (i % 100 == 0):
            sys.stdout.write("Reading %s: %d%%   \r" % (tree.GetName(),100*i/(maxEvents+skipEvents)) )
            sys.stdout.flush()
        if i >= (maxEvents+skipEvents):
            break
        selectedEvNum = eventList.GetEntry(i)
        tree.GetEntry(selectedEvNum)
        for feature in featureNames:
            dataContainer[feature].append(tree.__getattr__(feature))
    sys.stdout.write("\n")

    # Make the numpy arrays
    outputArray = np.array([])
    for feature in allowedFeatures:
        column = dataContainer[feature]
        feature_vector = np.asarray(column)
        feature_vector = feature_vector.reshape(feature_vector.size,1)
        if outputArray.shape[0]==0:
            outputArray = feature_vector
        else:
            outputArray = np.concatenate((outputArray,feature_vector),axis=1)
    imp = Imputer(missing_values=-999, strategy='mean', axis=0)
    imp.fit(outputArray)
    outputArray = imp.transform(outputArray)
    print(name)
    print("Events: ",outputArray.shape[0])
    print("Features: ",outputArray.shape[1])
    return outputArray

But if one want to play around with the same input many times it is probably better to write the numpy array into a h5 file once and for all and then read from this file (much faster than converting the ntuple). This can be done in the following function. It takes as input the ROOT file, name of tree in the ROOT file, which variable to write, the name of the output (with directory), the cut to use and finally the chunk size (i.e. how much is stored in memory for each iteration of writing to file)

In [ ]:
def writeArrays(fileName, treeName, ntupFeatures, outputFileName = "", selection = "", chunkSize = 1000000):
    # ROOT files
    print "Opening ROOT file..."
    if outputFileName:
        outputName = outputFileName+".h5"
    else:
        outputName = fileName[:len(fileName)-5]+".h5"

    inputFile = TFile(fileName)
    inputTree = inputFile.Get(treeName)

    # Chunks
    totalEvents = inputTree.GetEntries()
    nChunks = (totalEvents // chunkSize) + 1
    remainderSize = totalEvents-(nChunks-1)*chunkSize

    # Output file
    with h5py.File(outputName, "w") as f:

       # First chunk (numpy array)
       print "Processing events 0 to",chunkSize
       firstChunk = tree2array(inputTree,start=0,stop=chunkSize,step=1,selection=selection,branches=ntupFeatures)
       n = 0
       if firstChunk.shape[0] == 0: 
            while firstChunk.shape[0] == 0:
                firstChunk = tree2array(inputTree,start=chunkSize*n,
                                            stop=chunkSize*(n+1),step=1,selection=selection,branches=ntupFeatures)
                n += 1
                print "INFO \t Chunk is still empty, trying for the %i%s time "%(n,"nd" if n == 2 else ("rd" if n == 3 else ("st" if n == 1 else "th")))

       # Break into columns
       firstChunk = np.array(firstChunk[:].tolist())
        
       #print f.attrs.keys()
       #return 
       # Output data structure
       dtype = firstChunk.dtype
       row_count = firstChunk.shape[0]
       maxshape = (None,) + firstChunk.shape[1:]
       dset = f.create_dataset(treeName, shape=firstChunk.shape, maxshape=maxshape,chunks=firstChunk.shape, dtype=dtype)
       # Write the first chunk 
       dset[:] = firstChunk
       print type(dset)
       print dset.shape
       #print dset.attrs.create()

       # Loop over chunks
       totalEvents_written = firstChunk.shape[0]
       for chunk in range(chunkSize if n == 0 else chunkSize*n,nChunks*chunkSize,chunkSize):
          start = chunk
          stop = start+chunkSize 
          if (stop>totalEvents): stop = totalEvents 
          print "Processing events ",start," to ",stop      
          # Extraction of data in numpy form, chunk by chunk
          chunk = tree2array(inputTree,
                             start=start,
                             stop=stop,
                             step=1,
                             selection = selection,
                             branches=ntupFeatures)

          totalEvents_written += chunk.shape[0]
          # Break up into columns
          chunk = np.array(chunk[:].tolist())

          # Resize the dataset to accommodate the next chunk of rows
          dset.resize(row_count + chunk.shape[0], axis=0)

          # Write the next chunk
          dset[row_count:] = chunk

          # Increment the row count
          row_count += chunk.shape[0]

       # store the variable names for later
       f.attrs.create('var_names',ntupFeatures)
        
    print "Number of events            = ",totalEvents
    print "Number of events written    = ",totalEvents_written
    print "Number of features          = ",len(ntupFeatures)
    print "Output file name            = ",outputName

Next two cells open the root file with an retrieves the TTree. Then we get the name of all the variables available in the nTuple.

In [ ]:
# Input files and TTrees
inputFile = TFile("/mn/felt/u1/eirikgr/Downloads/TMVA_tree.root");
tree = inputFile.Get("TMVA_tree")
#inputFile = TFile("small.root");
#tree = inputFile.Get("TMVA_tree")

In [ ]:
susyFeaturesNtup = []
listofbr = tree.GetListOfBranches().Clone()
listofbr.SetOwner(False)
#listofbr.Sort()
for br in listofbr:
    susyFeaturesNtup.append(br.GetName())
print "List of the %i variables in this ntuple: %s" %(len(susyFeaturesNtup),", ".join(susyFeaturesNtup))

Let's convert the nTuple to a numpy array and write everything to a file. This will take a while:

In [ ]:
writeArrays("/mn/felt/u1/eirikgr/Downloads/TMVA_tree.root", "TMVA_tree", 
            susyFeaturesNtup, "/scratch3/eirikgr/ML_files/variables_all_bkg","isSignal==0", 100000)
writeArrays("/mn/felt/u1/eirikgr/Downloads/TMVA_tree.root", "TMVA_tree", 
            susyFeaturesNtup, "/scratch3/eirikgr/ML_files/variables_all_sig","isSignal==1", 100000)

# Testing some features on a small file

In [ ]:
#eventFile_bkg.close()
#eventFile_sig.close()
writeArrays("small.root", "TMVA_tree", 
            susyFeaturesNtup, "/scratch3/eirikgr/ML_files/variables_test_bkg","isSignal==0", 100)
writeArrays("small.root", "TMVA_tree", 
            susyFeaturesNtup, "/scratch3/eirikgr/ML_files/variables_test_sig","isSignal==1", 100)

Please note that some of these variables should not be used as discriminating variables (e.g. using the isSignal variable would not be fair to include :-)). Also the wgt, massSplit, dsid and isBkg should not be used. We will remove these later, before we start using the data. 

In [ ]:
X = buildArraysFromROOT(tree,susyFeaturesNtup,"",0,100,"ALL")
X[0]
eventFile = h5py.File("/scratch3/eirikgr/ML_files/variables_test.h5","r")
print eventFile.attrs['var_names']
X_test = eventFile["TMVA_tree"]
#susyFeaturesNtup.remove("massSplit")
#susyFeaturesNtup.remove("wgt")
#susyFeaturesNtup.remove("isBkg")
#susyFeaturesNtup.remove("dsid")
#eventFile.close()
print X_test[:].shape
toplot = X_test[:,49]
print toplot.shape
print len(toplot)

In [ ]:
plt.hist(toplot,bins=100);
plt.xlim(0,4);

Just for fun, before we start playing, let's plot the transverse momentum of the hardest lepton (i.e. lepton with the highest transverse momentum in the event) in each event.

In [ ]:
canvas = TCanvas("canvas","canvas",1)
h_lep1_pt = TH1F("h_lep1_pt","p_{T}^{lep1};p_{T} [GeV];Number of entries",200,0,2000)
tree.Draw("lep_pT1/1000>>h_lep1_pt")
h_lep1_pt.Draw("hist")
canvas.SetLogy()
canvas.Draw()

The nTuple contains a mix of signal and background samples. These can be identified by plotting the isSignal variable (equals 1 if event is signal, 0 if background)

In [ ]:
canvas_2 = TCanvas("canvas_2","canvas_2",1)
h_isSignal = TH1F("h_isSignal","isSignal;;Number of entries",2,0,2)
h_isSignal.GetXaxis().SetBinLabel(1,"Background")
h_isSignal.GetXaxis().SetBinLabel(2,"SUSY signal")
tree.Draw("isSignal>>h_isSignal")
h_isSignal.Draw("hist")
canvas_2.SetLogy()
canvas_2.Draw()

There are several signal samples in the nTuple (different SUSY scenarios). These are simplified models which means that they are basically simulations of a single feynman grap (see <a href=https://cds.cern.ch/record/2233683/files/ATL-COM-PHYS-2016-1673.pdf>here</a> for more details). The nTuple used here all signal samples are of the following kind
<img src="https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/SUSY/FeynmanGraphs/C1C1-llvvN1N1-slsnu.png",width=160,height=160>
where the only difference between the various signal models is the mass difference between the chargino and the neutralino. Let's look at the values for the mass splitting for the available signal models

In [ ]:
canvas_3 = TCanvas("canvas_3","canvas_3",1)
h_massSplit = TH1F("h_massSplit","M(#chi_{1})-M(#chi_{0});Mass split [GeV];Number of entries",400,0,800)
tree.Draw("massSplit>>h_massSplit", "isSignal==1")
h_massSplit.Draw("hist")
#canvas_3.Draw()
canvas_8 = TCanvas("canvas_8","canvas_8",1)
h_massSplit_dsid = TH2F("h_massSplit_dsid","M(#chi_{1}^{#pm})-M(#chi_{1}^{0});Mass split [GeV];Data set ID",400,0,800,22,392500,392521)
tree.Draw("dsid:massSplit>>h_massSplit_dsid", "isSignal==1")
h_massSplit_dsid.Draw("")
canvas_8.Draw()

Define how to divide background from signal

In [ ]:
# Selections
cutBackground = "isSignal==0"
cutSignal = "isSignal==1"

We also need to single out the event weight which makes sure all the background samples are scaled to the same luminosity

In [ ]:
susyWeightsNtup = [ "wgt" ]
susyIdentifierNtup = [ "isSignal" ]

In [ ]:
# Automatic numbers of event 
tcut = TCut(cutSignal)
tree.Draw(">>eventList",tcut)
eventList = TEventList()
eventList = gDirectory.Get("eventList")
nSignalEvents = eventList.GetN()/2
#nSignalEvents = 20000
nBackgroundEvents = nSignalEvents
print "Will use %i events for signal and %i for background" %(nSignalEvents,nBackgroundEvents)

In [ ]:
#canvas_4 = TCanvas("canvas_4","canvas_4",1)
#h_MET_signal = TH1F("h_MET_signal","Missing Transverse Energy;Missing Transverse Energy [GeV];Number of entries",500,0,1000)
#h_MET_backgr = TH1F("h_MET_backgr","Missing Transverse Energy;Missing Transverse Energy [GeV];Number of entries",500,0,1000)
#tree.Draw("met/1000.>>h_MET_signal", cutSignal)
#tree.Draw("met/1000.>>h_MET_backgr", cutBackground)
#h_MET_signal.SetLineColor(kRed)
#h_MET_signal.Scale(1./h_MET_signal.Integral())
#h_MET_backgr.Scale(1./h_MET_backgr.Integral())
#h_MET_backgr.Draw("hist")
#h_MET_signal.Draw("hist same")
#canvas_4.SetLogy()
#canvas_4.Draw()

In [ ]:
#canvas_5 = TCanvas("canvas_5","canvas_5",1)
#h_MEFF_MET_signal = TH2F("h_MEFF_MET_signal","Missing Transverse Energy vs. M_{eff};Missing Transverse Energy [GeV];M_{eff} [GeV]",500,0,1000,500,0,1000)
#h_MEFF_MET_backgr = TH2F("h_MEFF_MET_backgr","Missing Transverse Energy vs. M_{eff};Missing Transverse Energy [GeV];M_{eff} [GeV]",500,0,1000,500,0,1000)
#tree.Draw("met/1000.:meff/1000.>>h_MEFF_MET_signal", cutSignal)
#tree.Draw("met/1000.:meff/1000.>>h_MEFF_MET_backgr", cutBackground)
#h_MET_signal.SetMarkerColor(kRed)
#h_MET_signal.Scale(1./h_MET_signal.Integral())
#h_MET_backgr.Scale(1./h_MET_backgr.Integral())
#h_MET_backgr.Draw("")
#h_MET_signal.Draw("")
#canvas_5.SetLogy()
#canvas_5.Draw()

In [ ]:
# Build data arrays
# Background
X_train_bg = buildArraysFromROOT(tree,susyFeaturesNtup,cutBackground,0,nBackgroundEvents,"TRAINING SAMPLE (background)")
X_test_bg = buildArraysFromROOT(tree,susyFeaturesNtup,cutBackground,nBackgroundEvents,nBackgroundEvents,"TESTING SAMPLE (background)")

# Signal
X_train_sig = buildArraysFromROOT(tree,susyFeaturesNtup,cutSignal,0,nSignalEvents,"TRAINING SAMPLE (signal)")
X_test_sig = buildArraysFromROOT(tree,susyFeaturesNtup,cutSignal,nSignalEvents,nSignalEvents,"TESTING SAMPLE (signal)")

In [ ]:
writeArrays(, selection = "", chunkSize = 1000000)
X_train_bg.shape

# Logistic Regression

In [ ]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
#writeArrays("/mn/felt/u1/eirikgr/Downloads/TMVA_tree.root", "TMVA_tree", 
            #susyFeaturesNtup, "/scratch3/eirikgr/ML_files/variables_background",cutBackground, 1000000)
#X_bkg = buildArraysFromROOT(tree,susyFeaturesNtup,cutBackground,0,50,"Variable")
#Y_bkg = buildArraysFromROOT(tree,susyIdentifierNtup,cutBackground,0,50,"Signal or no signal")
#X_sig = buildArraysFromROOT(tree,susyFeaturesNtup,cutSignal,0,50,"Variable")
#Y_sig = buildArraysFromROOT(tree,susyIdentifierNtup,cutSignal,0,50,"Signal or no signal")

In [ ]:
#writeArrays("/mn/felt/u1/eirikgr/Downloads/TMVA_tree.root", "TMVA_tree", 
            #susyFeaturesNtup, "/scratch3/eirikgr/ML_files/variables_signal",cutSignal, 100000)

Now we're ready to read the numpy array from file and remove the variables we don't want to include as disriminating variables in our fit. 

In [ ]:
#backgroundFile = h5py.File("/scratch3/eirikgr/ML_files/variables_background.h5","r")
#signalFile     = h5py.File("/scratch3/eirikgr/ML_files/variables_signal.h5","r")
#eventFile = h5py.File("/scratch3/eirikgr/ML_files/variables_all.h5","r")
#print type(eventFile)
eventFile_bkg = h5py.File("/scratch3/eirikgr/ML_files/variables_test_bkg.h5","r")
eventFile_sig = h5py.File("/scratch3/eirikgr/ML_files/variables_test_sig.h5","r")
attributes = eventFile_bkg.attrs["var_names"]
#print attributes.shape
susyFeaturesExclude = {}
susyFeaturesExclude["massSplit"] = -1
susyFeaturesExclude["wgt"] = -1
susyFeaturesExclude["isBkg"] = -1
susyFeaturesExclude["isSignal"] = -1
susyFeaturesExclude["dsid"] = -1
#print itemindex[0][0]
delitems = []
for sfe in susyFeaturesExclude.keys():
    itemindex = np.where(attributes==sfe)
    print "Removing variable %-10s in index %i" %(sfe,itemindex[0][0])
    delitems.append(itemindex[0][0])
    susyFeaturesExclude[sfe] = itemindex[0][0]
# get the full numpy array
all_var_bkg = eventFile_bkg["TMVA_tree"]
all_var_sig = eventFile_sig["TMVA_tree"]
print "\n"
print "There are a total of %i events in the background nTuple" %(all_var_bkg.shape[0])
print "There are a total of %i events in the signal nTuple" %(all_var_sig.shape[0])
print "\n"
# get the signal/no-signal vector
Y_bkg = all_var_bkg[:,susyFeaturesExclude["isSignal"]]
Y_sig = all_var_sig[:,susyFeaturesExclude["isSignal"]]
# get the weights
W_bkg = all_var_bkg[:,susyFeaturesExclude["wgt"]]
W_sig = all_var_sig[:,susyFeaturesExclude["wgt"]]
# get the numpy array without the unwanted variables
X_bkg = np.delete(all_var_bkg,delitems,1)
X_sig = np.delete(all_var_sig,delitems,1)

print "Backgrounds are now stored in X_bkg with %i entries and %i variables" %(X_bkg.shape[0],X_bkg.shape[1])
print "Signals     are now stored in X_sig with %i entries and %i variables" %(X_sig.shape[0],X_sig.shape[1])
print "\n"
print "Target  for background is now stored in Y_bkg with %i entries" %(Y_bkg.shape[0])
print "Weights for background are now stored in W_bkg with %i entries" %(W_bkg.shape[0])
print "\n"
print "Target  for signals is now stored in Y_sig with %i entries" %(Y_sig.shape[0])
print "Weights for signals are now stored in W_sig with %i entries" %(W_sig.shape[0])

Get the number of events for each data set ID (dsid). We want to train on all the various backgrounds and signal.

In [ ]:
eventFile_bkg = h5py.File("/scratch3/eirikgr/ML_files/variables_all_bkg.h5","r")
eventFile_sig = h5py.File("/scratch3/eirikgr/ML_files/variables_all_sig.h5","r")
attributes = eventFile_sig.attrs["var_names"]
#print attributes.shape
susyFeaturesExclude = {}
susyFeaturesExclude["massSplit"] = -1
susyFeaturesExclude["wgt"] = -1
susyFeaturesExclude["isBkg"] = -1
susyFeaturesExclude["isSignal"] = -1
susyFeaturesExclude["dsid"] = -1
#print itemindex[0][0]
delitems = []
for sfe in susyFeaturesExclude.keys():
    itemindex = np.where(attributes==sfe)
    print "Unwanted variable %-10s in index %i" %(sfe,itemindex[0][0])
    delitems.append(itemindex[0][0])
    susyFeaturesExclude[sfe] = itemindex[0][0]
# get the full numpy array
all_var_bkg = eventFile_bkg["TMVA_tree"]
all_var_sig = eventFile_sig["TMVA_tree"]
print "\n"
print "There are a total of %i events in the background nTuple" %(all_var_bkg.shape[0])
print "There are a total of %i events in the signal nTuple" %(all_var_sig.shape[0])
print "\n"

Let's produce the data set to train on. Should be a mix of backgrounds and signals. 

In [ ]:
rBackgroundEvents = 0.2
rSignalEvents = 0.2
X_train_bkg = {}
X_train_sig = {}
X_test_bkg  = {}
X_test_sig  = {}
W_train_bkg = {}
W_train_sig = {}
W_test_bkg  = {}
W_test_sig  = {}
counts_dsids = {}

In [ ]:
dsids = np.concatenate((all_var_bkg[:,susyFeaturesExclude["dsid"]],all_var_sig[:,susyFeaturesExclude["dsid"]]),axis=0)
print "There are %i unique data sets in the signal and background samples" %np.unique(dsids).shape[0]
n = 0
for dsid in np.unique(dsids):
    #
    if (dsid >= 392500 and dsid <= 392521): continue
    print "Doing dsid %i/%i" %(n,np.unique(dsids).shape[0])
    num = 0
    isBkg = True
    indexs = np.where(all_var_bkg[:,susyFeaturesExclude["dsid"]] == dsid)
    num = indexs[0].shape[0]
    if not num: 
        indexs = np.where(all_var_sig[:,susyFeaturesExclude["dsid"]] == dsid)
        num = indexs[0].shape[0]
        isBkg = False
    if num < 5: 
        print "Skipping %i since only %i events" %(dsid,num)
        continue
    if isBkg:
        counts_dsids[dsid] = num
        X_train_bkg[dsid] = all_var_bkg[indexs[0][:int(math.ceil(num*rBackgroundEvents))],:]
        X_test_bkg[dsid]  = all_var_bkg[indexs[0][int(math.ceil(num*rBackgroundEvents)):],:]
    else:   
        num = indexs[0].shape[0]
        print num
        counts_dsids[dsid] = num
        X_train_sig[dsid] = all_var_sig[indexs[0][:int(math.ceil(num*rSignalEvents))],:]
        X_test_sig[dsid]  = all_var_sig[indexs[0][int(math.ceil(num*rSignalEvents)):],:]
        print X_train_sig[dsid].shape[0]
    n += 1
        
for dsid in sorted(counts_dsids.keys()):
    if dsid in X_train_bkg.keys():
        W_train_bkg[dsid] = X_train_bkg[dsid][:,susyFeaturesExclude["wgt"]]
        W_test_bkg[dsid]  = X_test_bkg[dsid][:,susyFeaturesExclude["wgt"]]
        print "DSID %i has %i events and %i (%i) events used for training (testing)" %(int(dsid),counts_dsids[dsid],X_train_bkg[dsid].shape[0],X_test_bkg[dsid].shape[0])
    elif dsid in X_train_sig.keys():
        W_train_sig[dsid] = X_train_sig[dsid][:,susyFeaturesExclude["wgt"]]
        W_test_sig[dsid]  = X_test_sig[dsid][:,susyFeaturesExclude["wgt"]]
        print "DSID %i has %i events and %i (%i) events used for training (testing)" %(int(dsid),counts_dsids[dsid],X_train_sig[dsid].shape[0],X_test_sig[dsid].shape[0])   

In [ ]:
W_train = np.array([])
X_train = np.array([])
W_test  = np.array([])
X_test  = np.array([])
n = 1
for dsid in sorted(counts_dsids.keys()):
    if dsid in X_train_bkg.keys():
        if X_train.shape[0] == 0:
            X_train = np.copy(X_train_bkg[dsid])
            W_train = np.copy(W_train_bkg[dsid])
            X_test = np.copy(X_test_bkg[dsid])
            W_test = np.copy(W_test_bkg[dsid])
        else:
            X_train = np.concatenate((X_train_bkg[dsid],X_train),axis=0)
            W_train = np.concatenate((W_train_bkg[dsid],W_train),axis=0)
            X_test  = np.concatenate((X_test_bkg[dsid],X_test),axis=0)
            W_test  = np.concatenate((W_test_bkg[dsid],W_test),axis=0)
        print "Adding %i/%i with %i events to training sample. Now is %i" %(n,len(counts_dsids.keys()),X_train_bkg[dsid].shape[0],X_train.shape[0])
    elif dsid in X_train_sig.keys():
        if X_train.shape[0] == 0:
            X_train = np.copy(X_train_sig[dsid])
            W_train = np.copy(W_train_sig[dsid])
            X_test  = np.copy(X_test_sig[dsid])
            W_test  = np.copy(W_test_sig[dsid])
        else:
            X_train = np.concatenate((X_train_sig[dsid],X_train),axis=0)
            W_train = np.concatenate((W_train_sig[dsid],W_train),axis=0)
            X_test  = np.concatenate((X_test_sig[dsid],X_test),axis=0)
            W_test  = np.concatenate((W_test_sig[dsid],W_test),axis=0)
        print "Adding %i/%i with %i events to training sample. Now is %i" %(n,len(counts_dsids.keys()),X_train_sig[dsid].shape[0],X_train.shape[0])
   
        
    n += 1

In [ ]:
# check that the shapes match and make the target vector (Y)
print W_train.shape
print X_train.shape
Y_train = np.where(X_train[:,susyFeaturesExclude["isSignal"]],1,0)
print Y_train.shape

In [ ]:
# check that the shapes match and make the target vector (Y)
print W_test.shape
print X_test.shape
Y_test = np.where(X_test[:,susyFeaturesExclude["isSignal"]],1,0)
print Y_test.shape

In [ ]:
X_train_clean = np.delete(X_train,delitems,1)
X_test_clean = np.delete(X_test,delitems,1)
print X_test_clean.shape
print "TRAIN sample: Removed %i variables (%s), shape is now (%s,%s)" %(len(delitems),",".join(susyFeaturesExclude),X_train_clean.shape[0],X_train_clean.shape[1])
print "TEST  sample: Removed %i variables (%s), shape is now (%s,%s)" %(len(delitems),",".join(susyFeaturesExclude),X_test_clean.shape[0],X_test_clean.shape[1])

In [ ]:
degree = 2
lambda_=1e-5
logreg = linear_model.LogisticRegression(C=1/lambda_,max_iter=200)

In [ ]:
logreg.fit(X_train_clean, Y_train.ravel(),W_train)

In [ ]:
sorted_coeff = np.sort(logreg.coef_[0])
i = 1
data = np.arange(len(sorted_coeff),dtype="f")
label = np.arange(len(sorted_coeff),dtype='object')
for sc in reversed(sorted_coeff):
    itemindex = np.where(logreg.coef_[0]==sc)
    label[i-1] = attributes[itemindex[0][0]]
    data[i-1] = float(sc)
    i += 1
plt.figure(figsize=(38,20))
plt.xticks(rotation='vertical',fontsize=42)
plt.bar(np.arange(i-1),data,tick_label=label,bottom=1e-8)

In [ ]:
def print_score(X,Y,logreg):
    predictions=logreg.predict(X)
    print('Accuracy:', accuracy_score(Y, predictions))
    print('Precision:', precision_score(Y, predictions, average='macro'))
    print('Recall:', recall_score(Y, predictions, average='macro'))
    return predictions

In [ ]:
pred = print_score(X_train_clean, Y_train,logreg)

In [ ]:
X_test_clean = np.delete(X_test,delitems,1)
print_score(X_test_clean, Y_test,logreg)

In [ ]:
pred = logreg.predict(X_train)
dsid      = np.concatenate((all_var_bkg[:,50],all_var_sig[:,50]),axis=0)
masssplit = np.concatenate((all_var_bkg[:,53],all_var_sig[:,53]),axis=0)
wgt       = np.concatenate((W_bkg,W_sig),axis=0)
predRight = 0.
predWrong = 0.
yields    = {}
label = np.arange(len(sorted_coeff),dtype='object')
for i in range(len(pred)):
    if not dsid[i] in yields.keys():
        yields[dsid[i]] = {"nbkg":0,"nsig":0}
    if pred[i]: yields[dsid[i]]["nsig"] += wgt[i]
    else: yields[dsid[i]]["nbkg"] += wgt[i]
    #print "Predicted %i, true is %i" %(pred[i],Y_test[i])
    if pred[i] == Y_train[i]:
        predRight += 1.
    else:
        predWrong += 1.
print "Got it right in %.2f of the cases!" %(predRight/float(i))
print yields

In [ ]:
h_MET_sig = TH1F("h_MET_sig","",100,0,1000)
h_MET_bkg = TH1F("h_MET_bkg","",100,0,1000)
n = 0
for p in pred:
    if p == 0:
        h_MET_bkg.Fill(X_test[n][19]/1000.)
    if p == 1:
        h_MET_sig.Fill(X_test[n][19]/1000.)
    n += 1
    
canvas_6 = TCanvas("canvas_6","canvas_6",1)
h_MET_sig.SetLineColor(kRed)
h_MET_bkg.Draw("hist")
h_MET_sig.Draw("same hist")
canvas_6.SetLogy()
canvas_6.Draw()

In [ ]:
logreg.predict_proba(X_test)

In [ ]:
def plot_boundary(X,y,degree,logreg):
    x_min, x_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    y_min, y_max = X[:, 2].min() - .5, X[:, 2].max() + .5
    xs, ys = np.meshgrid(np.linspace(x_min, x_max, 200),np.linspace(y_min, y_max, 200))
    xys = mapFeature(xs.ravel(),ys.ravel(),degree)
    Z = logreg.predict(xys).reshape(xs.shape)
    plt.pcolormesh(xs, ys, Z, cmap=plt.cm.Paired)
    plt.scatter(X[:, 1], X[:, 2], c=Y, edgecolors='k', cmap=plt.cm.Paired)
    plt.xlim(xs.min(), xs.max())
    plt.ylim(ys.min(), ys.max())
    plt.xticks(())
    plt.yticks(())
    return plt

In [ ]:
def mapFeature(X1, X2, degree):
    out = np.ones(( X1.shape[0], sum(range(degree + 2)) )) # could also use ((degree+1) * (degree+2)) / 2 instead of sum
    curr_column = 1
    for i in range(1, degree + 1):
        for j in range(i+1):
            out[:,curr_column] = np.power(X1,i-j) * np.power(X2,j)
            curr_column += 1
    return out

In [ ]:
import matplotlib.pyplot as plt
plt=plot_boundary(X,Y,degree,logreg)
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()